# 04. Model Workflow

In [4]:
library(tidyverse)
library(tidymodels)
ggplot2::theme_set(theme_bw())
tidymodels_prefer()

<br>

## 04.01. 모델의 시작과 끝
- 일부 간단한 데이터 세트의 경우, 모델 자체를 적합하는 것이 프로세스의 전부일 수 있음
- 하지만 모델이 적합되기 전, 다양한 선택과 추가 단계가 필요한 경우가 존재
    - 예1) 모델링에 $p$개의 설명변수를 사용할 때, 일반적으로 $p$보다 많은 개수의 변수가 존재
    
        $\rightarrow$ 탐색적 데이터 분석이나 도메인 지식을 활용하여 일부 예측 변수가 분석에서 제외        
        
    <br>
    
    - 예2) 결측값에 대한 전처리
    - 예3) PCA와 같은 설명 변수로 새로운 변수를 생성

<br>

- 이러한 예는 모델이 적합하기 전에 발생하는 단계와 관련되어 있지만 모델이 생성된 후에 발생하는 작업도 있을 수 있음

<br>

- **매개변수를 추정하는데 사용되는 특정 모델만 맞추는 것이 아니라, 더 넓은 모델링 프로세스에 집중하는 것이 중요**

    **전처리 단계, 모델 적합성 및 잠재적인 후처리 활동이 포함**
    
    $\rightarrow$ **모델 워크폴로**

<br>

### PCA(주성분 분석) 예시

<br>

- **PCA 단계를 모델링 작업 흐름의 일부가 아닌 것으로 간주** $\rightarrow$  **잘못된 모델**
    - 모델 워크플로에 PCA가 포함되지 않음으로써, PCA 효과를 적절하게 측정할 수 없음




<br>

- **적절한 모델 : PCA 단계를 모델링 작업 흐름의 일부로 간주**



<br>

## 04.02. 모델 워크플로 기본
- **모델링 프로세스의 주요 부분을 객체화**

In [5]:
ames <- ames %>% mutate(Sale_Price = log10(Sale_Price))
set.seed(20221010)
ames_split <- initial_split(ames, prop = 0.8, strata = Sale_Price)
ames_train <- training(ames_split)
ames_test <- testing(ames_split)

- 모델 생성

In [6]:
lm_model <- linear_reg() %>% set_engine("lm")

- 워크플로 객체 생성 및 모델 추가

In [7]:
lm_wflow <- workflow() %>%
    add_model(lm_model)

lm_wflow

══ Workflow ════════════════════════════════════════════════════════════════════════════════════════════════════════════
Preprocessor: None
Model: linear_reg()

── Model ───────────────────────────────────────────────────────────────────────────────────────────────────────────────
Linear Regression Model Specification (regression)

Computational engine: lm 


- 모델 공식 추가

In [8]:
lm_wflow <- lm_wflow %>%
    add_formula(Sale_Price ~ Longitude + Latitude)

- 데이터 적합

In [9]:
lm_fit <- fit(lm_wflow, ames_train)
lm_fit

══ Workflow [trained] ══════════════════════════════════════════════════════════════════════════════════════════════════
Preprocessor: Formula
Model: linear_reg()

── Preprocessor ────────────────────────────────────────────────────────────────────────────────────────────────────────
Sale_Price ~ Longitude + Latitude

── Model ───────────────────────────────────────────────────────────────────────────────────────────────────────────────

Call:
stats::lm(formula = ..y ~ ., data = data)

Coefficients:
(Intercept)    Longitude     Latitude  
   -307.997       -2.042        2.901  


- 예측

In [10]:
predict(lm_fit, ames_test %>% slice(1:3))

.pred
<dbl>
5.284402
5.245254
5.238754


- 워크플로 제거 / 수정

In [11]:
lm_fit %>% update_formula(Sale_Price ~ Longitude)

══ Workflow ════════════════════════════════════════════════════════════════════════════════════════════════════════════
Preprocessor: Formula
Model: linear_reg()

── Preprocessor ────────────────────────────────────────────────────────────────────────────────────────────────────────
Sale_Price ~ Longitude

── Model ───────────────────────────────────────────────────────────────────────────────────────────────────────────────
Linear Regression Model Specification (regression)

Computational engine: lm 


<br>

## 04.03. 모델 변수 선택

<br>

#### ```add_variables(outcome, predictors)```
- ```outcome``` : 반응변수
- ```predictors``` : 설명변수
    - ```ends_with()```를 사용하여 지정 가능
    - ```everything()``` : 모든 변수

<br>

- 워크플로 객체 수식 초기화 및 변수 재설정

In [12]:
lm_wflow <- lm_wflow %>%
    remove_formula() %>%
    add_variables(outcome = Sale_Price, predictors = ends_with('tude'))

lm_wflow

══ Workflow ════════════════════════════════════════════════════════════════════════════════════════════════════════════
Preprocessor: Variables
Model: linear_reg()

── Preprocessor ────────────────────────────────────────────────────────────────────────────────────────────────────────
Outcomes: Sale_Price
Predictors: ends_with("tude")

── Model ───────────────────────────────────────────────────────────────────────────────────────────────────────────────
Linear Regression Model Specification (regression)

Computational engine: lm 


- 데이터 적합

In [13]:
fit(lm_wflow, ames_train) 

══ Workflow [trained] ══════════════════════════════════════════════════════════════════════════════════════════════════
Preprocessor: Variables
Model: linear_reg()

── Preprocessor ────────────────────────────────────────────────────────────────────────────────────────────────────────
Outcomes: Sale_Price
Predictors: ends_with("tude")

── Model ───────────────────────────────────────────────────────────────────────────────────────────────────────────────

Call:
stats::lm(formula = ..y ~ ., data = data)

Coefficients:
(Intercept)    Longitude     Latitude  
   -307.997       -2.042        2.901  


<br>

## 04.04. Workflow 수식

<br>

### 특수 수식 및 인라인 함수
- 다수의 다중 레벨 모델이 고안된 `lme4` 패키지 
- 예) 피험자에게 랜덤효과가 있는 회귀 모델을 적합
    
    $\rightarrow$ 성별마다 각기 다른 절편 및 매개변수 추정량을 가지게 됨

In [19]:
library(lme4)
library(nlme)

data(Orthodont)

In [28]:
lmer(distance ~ Sex + (age | Subject), data = Orthodont) %>% summary

Linear mixed model fit by REML ['lmerMod']
Formula: distance ~ Sex + (age | Subject)
   Data: Orthodont

REML criterion at convergence: 471.2

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.8941 -0.3149  0.0530  0.3673  3.8564 

Random effects:
 Groups   Name        Variance Std.Dev. Corr 
 Subject  (Intercept) 54.630   7.3912        
          age          0.482   0.6943   -0.97
 Residual              1.716   1.3100        
Number of obs: 108, groups:  Subject, 27

Fixed effects:
            Estimate Std. Error t value
(Intercept)  24.5170     0.4835  50.710
SexFemale    -2.1455     0.7575  -2.832

Correlation of Fixed Effects:
          (Intr)
SexFemale -0.638

<br>

### 계층적 회귀모델
#### `multilevelmod`

In [21]:
library(multilevelmod)

- 모델 생성

In [22]:
multilevel_spec <- linear_reg() %>% set_engine("lmer")

- 워크플로 생성 및 모델 추가

In [23]:
multilevel_workflow <- workflow() %>%
    add_variables(outcome = distance, predictors = c(Sex, age, Subject)) %>%
    add_model(multilevel_spec, formula = distance ~ Sex + (age | Subject))

- 데이터 적합

In [24]:
multilevel_fit <- fit(multilevel_workflow, data = Orthodont)
multilevel_fit

══ Workflow [trained] ══════════════════════════════════════════════════════════════════════════════════════════════════
Preprocessor: Variables
Model: linear_reg()

── Preprocessor ────────────────────────────────────────────────────────────────────────────────────────────────────────
Outcomes: distance
Predictors: c(Sex, age, Subject)

── Model ───────────────────────────────────────────────────────────────────────────────────────────────────────────────
Linear mixed model fit by REML ['lmerMod']
Formula: distance ~ Sex + (age | Subject)
   Data: data
REML criterion at convergence: 471.1635
Random effects:
 Groups   Name        Std.Dev. Corr 
 Subject  (Intercept) 7.3912        
          age         0.6943   -0.97
 Residual             1.3100        
Number of obs: 108, groups:  Subject, 27
Fixed Effects:
(Intercept)    SexFemale  
     24.517       -2.145  

<br>

### 생존함수

In [51]:
library(censored)

In [52]:
parametic_spec <- survival_reg()

In [53]:
parametic_workflow <-
    workflow() %>%
    add_variables(outcome = c(fustat, futime), predictors = c(age, rx)) %>%
    add_model(parametic_spec,
              formula = Surv(futime, fustat) ~ age + strata(rx))

In [54]:
parametic_fit <- fit(parametic_workflow, data = ovarian)
parametic_fit

== Workflow [trained] ==================================================================================================
Preprocessor: Variables
Model: survival_reg()

-- Preprocessor --------------------------------------------------------------------------------------------------------
Outcomes: c(fustat, futime)
Predictors: c(age, rx)

-- Model ---------------------------------------------------------------------------------------------------------------
Call:
survival::survreg(formula = Surv(futime, fustat) ~ age + strata(rx), 
    data = data, model = TRUE)

Coefficients:
(Intercept)         age 
 12.8734120  -0.1033569 

Scale:
     rx=1      rx=2 
0.7695509 0.4703602 

Loglik(model)= -89.4   Loglik(intercept only)= -97.1
	Chisq= 15.36 on 1 degrees of freedom, p= 8.88e-05 
n= 26 

<br>

## 04.05. 복수 워크플로 생성


<br>

#### ```workflow_set(preproc, models)```
- ```preproc``` : 수식 List
- ```models``` : 적합할 모형식

<br>

- 수식 List 생성

In [30]:
location <- list(
    Longitude = Sale_Price ~ Longitude,
    Latitude = Sale_Price ~ Latitude,
    coords = Sale_Price ~ Longitude + Latitude,
    neighborhood = Sale_Price ~ Neighborhood
)

- 모델 생성

In [35]:
lm_model <- linear_reg() %>% set_engine("lm")

<br>

- 워크플로 세트 생성

In [33]:
library(workflowsets)

In [32]:
location_models <- workflow_set(preproc = location, models = list(lm = lm_model))
location_models

wflow_id,info,option,result
<chr>,<list>,<list>,<list>
Longitude_lm,"Sale_Price ~ Longitude, ~NULL, ~NULL, regression, FALSE, lm, TRUE, FALSE, formula, linear_reg,",NULL,NULL
Latitude_lm,"Sale_Price ~ Latitude, ~NULL, ~NULL, regression, FALSE, lm, TRUE, FALSE, formula, linear_reg,",NULL,NULL
coords_lm,"Sale_Price ~ Longitude + Latitude, ~NULL, ~NULL, regression, FALSE, lm, TRUE, FALSE, formula, linear_reg,",NULL,NULL
neighborhood_lm,"Sale_Price ~ Neighborhood, ~NULL, ~NULL, regression, FALSE, lm, TRUE, FALSE, formula, linear_reg,",NULL,NULL


<br>

- 모델 추출
#### ```extract_workflow()``` : workflow_set에서 모델 추출

In [36]:
extract_workflow(location_models, id = "coords_lm")

══ Workflow ════════════════════════════════════════════════════════════════════════════════════════════════════════════
Preprocessor: Formula
Model: linear_reg()

── Preprocessor ────────────────────────────────────────────────────────────────────────────────────────────────────────
Sale_Price ~ Longitude + Latitude

── Model ───────────────────────────────────────────────────────────────────────────────────────────────────────────────
Linear Regression Model Specification (regression)

Computational engine: lm 


In [38]:
location_models <-  location_models %>%
    mutate(fit = map(info, ~ fit(.x$workflow[[1]], ames_train)))

In [40]:
location_models$fit[[1]]

══ Workflow [trained] ══════════════════════════════════════════════════════════════════════════════════════════════════
Preprocessor: Formula
Model: linear_reg()

── Preprocessor ────────────────────────────────────────────────────────────────────────────────────────────────────────
Sale_Price ~ Longitude

── Model ───────────────────────────────────────────────────────────────────────────────────────────────────────────────

Call:
stats::lm(formula = ..y ~ ., data = data)

Coefficients:
(Intercept)    Longitude  
   -178.068       -1.957  


<br>

## 04.06. 테스트 세트 평가

<br>

#### ```last_fit()``` : 최종 모델을 훈련 세트에 적합, 테스트 세트로 평가

In [45]:
ames_split

<Training/Testing/Total>
<2342/588/2930>

In [49]:
final_lm_res <- last_fit(lm_wflow, ames_split)
print(final_lm_res)

# Resampling results
# Manual resampling 
# A tibble: 1 × 6
  splits             id               .metrics .notes   .predictions .workflow 
  <list>             <chr>            <list>   <list>   <list>       <list>    
1 <split [2342/588]> train/test split <tibble> <tibble> <tibble>     <workflow>


In [54]:
fitted_lm_wflow <- extract_workflow(final_lm_res)
fitted_lm_wflow

══ Workflow [trained] ══════════════════════════════════════════════════════════════════════════════════════════════════
Preprocessor: Variables
Model: linear_reg()

── Preprocessor ────────────────────────────────────────────────────────────────────────────────────────────────────────
Outcomes: Sale_Price
Predictors: ends_with("tude")

── Model ───────────────────────────────────────────────────────────────────────────────────────────────────────────────

Call:
stats::lm(formula = ..y ~ ., data = data)

Coefficients:
(Intercept)    Longitude     Latitude  
   -307.997       -2.042        2.901  


<br>

#### ```collect_metrics()``` : 성능 메트릭에 대한 정보 제공
#### ```collect_predictions()``` : 예측 메트릭에 대한 정보 제공

In [55]:
collect_metrics(final_lm_res)
collect_predictions(final_lm_res) %>% slice(1:5)

.metric,.estimator,.estimate,.config
<chr>,<chr>,<dbl>,<chr>
rmse,standard,0.1742739,Preprocessor1_Model1
rsq,standard,0.1582418,Preprocessor1_Model1


id,.pred,.row,Sale_Price,.config
<chr>,<dbl>,<int>,<dbl>,<chr>
train/test split,5.284402,7,5.329398,Preprocessor1_Model1
train/test split,5.245254,25,5.175802,Preprocessor1_Model1
train/test split,5.238754,30,4.982271,Preprocessor1_Model1
train/test split,5.238857,31,5.023252,Preprocessor1_Model1
train/test split,5.302665,50,5.311754,Preprocessor1_Model1
